In [ ]:
from bs4 import BeautifulSoup
import requests

# similar to our API requests, but now we just want all the text
html = requests.get('https://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_2021').text
soup = BeautifulSoup(html, 'html.parser')
print(soup.prettify())
table = soup.find('table', {"class":'wikitable sortable'})

In [ ]:
import pandas as pd

Wikipedia pages

In [ ]:
years = [2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012]

billboard = pd.DataFrame()
for year in years:
    url = 'https://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_' + str(year)
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'html.parser')
    #print(soup.prettify())
    table = soup.find('table', {"class":'wikitable sortable'})
    df = pd.read_html(str(table))
    df = pd.concat(df)
    df['Year'] = str(year)
    billboard = pd.concat([billboard, df], ignore_index=True)
    
billboard
billboard.to_csv("billboard.csv")

Spotify API

In [ ]:
CLIENT_ID = "914d599079cf414bb7441d9c09e8dd1d"
CLIENT_SECRET = "c67d0eee9ade4d49a710acf1a7615a3d"

AUTH_URL = "https://accounts.spotify.com/api/token"
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

#Convert response to JSON
auth_response_data = auth_response.json()

#Save the access token
access_token = auth_response_data['access_token']

#Need to pass access token into header to send properly formed GET request to API server
headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

In [ ]:
name = []
explicit = []
images = []
release_date = []
preview_url = []

In [ ]:
for index in billboard.index:
    title = billboard["Title"][index]
    artists = billboard["Artist(s)"][index]
    link = BASE_URL + 'search?q=' + title + '&type=track'
    r = requests.get(link, headers=headers)
    r = r.json()
    name.append(r["tracks"]['items'][0]["name"])
    explicit.append(r["tracks"]['items'][0]["explicit"])
    images.append(r["tracks"]['items'][0]['album']["images"][0]['url'])
    release_date.append(r["tracks"]['items'][0]['album']["release_date"])
    preview_url.append(r["tracks"]['items'][0]["preview_url"])

In [ ]:
name.append(r["tracks"]['items'][0]["name"])
explicit.append(r["tracks"]['items'][0]["explicit"])
images.append(r["tracks"]['items'][0]['album']["images"][0]['url'])
release_date.append(r["tracks"]['items'][0]['album']["release_date"])
preview_url.append(r["tracks"]['items'][0]["preview_url"])

In [ ]:
billboard.to_csv("bill_spo.csv")

iTunes API

In [ ]:
name_a = []
price = []
genre = []
trackTime = []
trackView = []

In [ ]:
for index in billboard.index:
    title = billboard["Title"][index].replace('"', '')
    artists = billboard["Artist(s)"][index]
    link = "https://itunes.apple.com/search?term=" + re.sub('[^a-zA-Z0-9 \n\.]', '', title)
    r = requests.get(link)
    r = r.json()
    name_a.append(r["results"][0]["trackName"])
    #price.append(r["results"][0]["trackPrice"])
    genre.append(r["results"][0]['primaryGenreName'])
    trackTime.append(r["results"][0]["trackTimeMillis"])
    trackView.append(r["results"][0]["trackViewUrl"])
    time.sleep(3)

In [ ]:
billboard['itunes_name'] = name_a
billboard['genre'] = genre
billboard['trackTime'] = trackTime
billboard['trackView'] = trackView

In [ ]:
billboard.to_csv("bill_spo_itunes.csv")